## Code Coffee: 17.05.22

# Yet another python accelerator: An introduction to ***numba***

We will take a look at the numba module which tries to achieve C-style performance for Python/Numpy based algorithms by compiling python into equivalent machine code on runtime. We will cover basic usage and limitations while appreciating significant speedup with minimal changes to existing code.  

The very good, but sometimes short ***numba*** documentation is available here
> https://numba.pydata.org/numba-doc/dev/index.html

There is also a nice video by "Jack of Some" on YouTube which is very easy to follow and explains basic usage and some pitfalls with ***numba***
> https://www.youtube.com/watch?v=x58W9A2lnQc

This introduction is inspired by both ;)  

## Installation

Its as easy as it can be and is even available in Anaconda/Miniconda
> pip install numba 

> conda install numba

In [1]:
# Basic modules we are going to use
import numpy as np
import numba as nb
from numba import jit, njit, vectorize

In [17]:
%timeit _ = sum_np(inp)

%timeit _ = sum_nb(inp)

%timeit _ = sum_nb2(inp)

%timeit _ = sum_nb3(lol)

1.03 s ± 34.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.83 s ± 39.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.78 s ± 19.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.73 s ± 17.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## The *@jit* decorator

Arguably the most important function decorator from the numba module. It enables the numba intrinsic compiler to optimize the decorated function and taylors it to the CPU architecture. 

In short, it takes the python generated bytecode of the decorated function and manages the type inferrence and compilation through LLVM (a compiler infrastructure project with lots of support for different CPU architectures and programming languages). 

It specifically compiles "just in time" for the CPU/GPU architecture you have. There is no need to make specific type definitions, and usually, it just works. 

In [ ]:
# Some funky behavior
def useless():
    pass

@jit
def what():
    pass

In [ ]:
%time useless()
%time what()

We can see that jitting an empty function slows down the execution.
This is because of the overhead caused by passing the bytecode generated by the python interpreter to the LLVM libraries where an empty function has to be translated. Executing the same function for a second time shows that the overhead vanishes since the function is now compiled.

#### Now for some test cases

In [ ]:
# Big data
test_array = np.arange(1_000_000)
test_list  = range(1_000_000)

def function_py(inp_list):
    """
    Pure python implementation.
    NEVER DO THIS!
    """
    out_list = []
    for val in inp_list:
        out_list.append(np.cos(val)**2 + np.sin(val)**2)
    return out_list

def function_np(inp):
    """
    Making use of numpy ufuncs.
    Numpy has the ability to broadcast single
    operations on entire arrays by itself.
    """
    return np.cos(inp)**2 + np.sin(inp)**2


In [ ]:
%time _ = function_py(test_list)
%time _ = function_np(test_array)

#### Numpy's vectorized ufuncs are good, but it can be better

In [ ]:
@jit
def lazy_function(inp):
    """
    The lazy compilation option.
    Just write @jit before the function.
    """
    return np.cos(inp)**2 + np.sin(inp)**2

In [ ]:
%time _ = lazy_function(test_array)

We see in this case, that the first time compilation is slower than the pure numpy implementation.  
This is again because of the compiler overhead.  
Interestingly, if we increase the array size, the *jitted* function eventually becomes faster even at first call.  

You can also explicitely tell numba, what it should expect as datatypes using a decorator signature:

In [ ]:
@jit(nb.float64[:](nb.float64[:]))
def eager_function(inp):
    """
    Eager compilation option.
    You can add a function signature to
    the @jit decorator. The function will be
    compiled expecting these datatypes as input
    and output. Other datatypes will not be
    accepted. This can be quite tricky...
    """
    return np.cos(inp)**2 + np.sin(inp)**2

In [ ]:
%time _ = eager_function(test_array)

If you are using signatures, you have to make sure that you pass the correct datastructures into the function.  
Otherwise it will give **errors** or show funky behavior.  
The advantage is, that the time of first compilation is saved, since ***numba*** already knows what to expect.  
A list of specifications for signatures can be found in the documentation:
- `void` is the return type of functions returning nothing (which actually return None when called from Python)
- `intp` and `uintp` are pointer-sized integers (signed and unsigned, respectively)
- `intc` and `uintc` are equivalent to C int and unsigned int integer types
- `int8`, `uint8`, `int16`, `uint16`, `int32`, `uint32`, `int64`, `uint64` are fixed-width integers of the corresponding bit width (signed and unsigned)
- `float32` and `float64` are single- and double-precision floating-point numbers, respectively
- `complex64` and `complex128` are single- and double-precision complex numbers, respectively
- array types can be specified by indexing any numeric type, e.g. `float32[:]` for a one-dimensional single-precision array or `int8[:,:]` for a two-dimensional array of 8-bit integers.

Signature syntax follows: `out_type(in_type_1, in_type_2, ...)`  

You can also set a list of signatures to 'overload' the function compilation.
Always order from least specific to most specific:

`int32 > int64 > float32 > float64`

otherwise unexpected behaviour might occur


In [ ]:
@jit([nb.float64[:](nb.int64[:]),
      nb.float64[:](nb.float64[:])])
def eager_function(inp):
    """
    Multiple signatures for a single
    function. Allows for various type
    definitions. If  you intend to use
    the jitted function with multiple
    input types, its easier to let numba
    figure it out, instead of wrting all
    definitions yourself...
    """
    return np.cos(inp)**2 + np.sin(inp)**2

### Some stupid example

Here, we will look at an example where jitting doesn't help

In [ ]:
def some_function(inp_list):
    """
    This example is directly taken from 'Jack of Some'
    on YouTube. It is however a nice illustration, how
    to NOT use numba
    """
    out_list = []
    for val in inp_list:
        if val % 2 == 0:
            out_list.append(2)
        else:
            out_list.append('1')
    return out_list


In [ ]:
%time _ = some_function(test_list)

In [ ]:
jit_some_function = jit()(some_function)

In [ ]:
%time _ = jit_some_function(test_list)

The *@jit* decorator can operate in two modes: *object mode* and *nopython mode*  
In *object mode* numba will fallback to the standard python behavior of dynamic datatypes, in addition to the compilation overhead.  
It is best to use the ***nopython*** option, as this will give **errors** instead of warnings, if *object mode* is invoked to resolve compilation.

The shorthand for *nopython* jitting is `njit`  


`jit(nopython=True) == njit()`

In [ ]:
njit_some_function = njit()(some_function)

In [ ]:
%time _ = njit_some_function(test_list)

The function failed in nopython mode because a string can't be added to a integer list.
This shows that numba requires type stability for datastructures like lists.  
Python lists are generally problematic, since ***numba*** has to do extra work (list reflection)  
to make sure the type of list entries is stable while going through the list.  
Additional problems arise from not knowing how large the `out_list` will be, thus requiring extra time.

Numba developers included new methods of dealing with lists by introducing 'typed.List' which can be imported from the numba module.  
It is derived from python lists but has a fixed type:

    import numba as nb
    mylist = nb.typed.List()

For the same reason, dictionaries and sets are only partially supported, but there should be typed versions of those as well, soon

If you have the option, ALWAYS use `numpy` arrays

### Let's fix the function

In [ ]:
def some_function_fix(inp_arr):
    """
    This example is directly taken from 'Jack of Some'
    on YouTube. This is how the function should look like,
    IF you intend to use @njit
    """
    # replace empty list with pre-allocated array
    # Careful with datatypes inside the function
    # when using signatures !!!
    out_arr = np.zeros_like(inp_arr)

    # enumerate is supported by numba
    for ii,val in enumerate(inp_arr):
        if val % 2 == 0:
            out_arr[ii] = 2
        else:
            out_arr[ii] = 1
    return out_arr

njit_some_function_fix = njit()(some_function_fix)

In [ ]:
%time _ = some_function_fix(test_array)
%time _ = njit_some_function_fix(test_array)

Because of the `enumerate` function, we explicitely require arrays as input to this function.  
If we define an output array inside a *jitted* function and use signatures, we have to make sure that
the datatype is the one we specified.  

Not all python/numpy features are yet supported and there are way too many features to cover here.  
Please refer to the official numba documentation for currently supported features.  
For example, support for `np.logspace` was only added recently (as of **13th Jan. 2022**)

Things like `try`-`except` or `raise`, `assert` are partially supported and limited in functionality as of yet.
Check the documentation for specifics. Thus it might be advantageous to keep ***numba*** *up-to-date* if you,  
even if some legacy features (reflected lists) might be deprecated in future releases.


## The ***@vectorize*** decorator

Similar to how numpy is able to apply scalar functions elementwise on entire arrays,  
***numba*** comes with a similar option: Instead of explicitely passing arrays to function calls  
and then explicitely loop over them, you can add the `@vectorize` decorator.  
It will tell the compiler to implicitely allow for array inputs and apply the function elementwise.  
Both methods (explicit and implicit) yield similar performance. Ultimately, it comes down to preference.

In [ ]:
def same_function_np(inp):
    """
    This example is directly taken from 'Jack of Some'
    on YouTube. This is how the function should look like.
    We will use numpy's own vectorize function
    -----
    Write 2 if inp is even, 1 if uneven
    """
    if inp % 2 == 0:
        return 2
    else:
        return 1

@vectorize(nopython=True)
def same_function(inp):
    """
    This example is directly taken from 'Jack of Some'
    on YouTube. This is how the function should look like,
    IF you intend to use @vectorize
    -----
    Write 2 if inp is even, 1 if uneven
    """
    if inp % 2 == 0:
        return 2
    else:
        return 1

In [ ]:
%time _ = same_function(test_array)

***numpy*** provides its own `vectorize` option which can create *ufuncs* for functions with branching  
return values.

In [ ]:
%time _ = np.vectorize(same_function_np)(test_array)

Notice, how much slower the ***numpy*** vectorization option is.  
This is due to `np.vectorize` being a wrapper function for an explicit loop  
over array elements. From the ***numpy*** documentation:
> The `(np.)vectorize` function is provided primarily for convenience, not for performance. The implementation is essentially a for loop.  

Since ***numba*** compiles its generated ufuncs, they achive similar speeds to ufuncs written in *C*

There is also the ***@guvectorize*** decorator (generalized ufuncs), which is similar to ***@vectorize*** but is intended  
for inplace manipulation of a result array given to the function:

In [ ]:
out_array = np.zeros_like(test_array)
scalar = 1.

@nb.guvectorize('(n),()->(n)')
def g(x, y, res):
    """
    Notice the special signature in the decorator.
    (n)     : 1D-array input
    ()      : scalar input
    ->(n)   : 1D-array output
    """
    for i in range(x.shape[0]):
        res[i] = x[i] + y

In [ ]:
%time g(test_array,scalar,out_array)

## More sensible examples

We will explore a classic benchmark test: calculating $\pi$ using a Monte-Carlo simulation.  
Additionally, I will show an example where I use ***numba*** it in my own research.

### Example functions I use with numba

Here are some examples where I use numba to gain some performance

In [12]:
@njit
def _sample(samp_arr: np.ndarray, CDF: np.ndarray, N: int) -> np.ndarray:
    """
    MC sampling of pseudo-CDF. Associates luminosities to population of XRBs 
    according to a given CDF. LumArr and CDF should have the same length
    -----
    samp_arr    :   Input array to sample value from (e.g. luminosity or energy)
    CDF         :   Input CDF derived from model and samp_arr (e.g. XLFs or spectra). 
                    Should have the same length as LumArr
    N           :   Population/Sample size. How often to sample from samp_arr.
                    Needs to be integer.
    """
    
    jj = 1
    kk = 1
    lum = np.zeros(N)
    ranvec = np.sort( np.random.rand(N) )
    
    for ii in range(0,N):
        jj = kk     # restart from jj where it arrived previously
        if jj == len(CDF) - 1:
            break   # otherwise, loop produces error due to overcounting
        while ( CDF[jj] < ranvec[ii] ):
            jj +=1
            if jj == len(CDF) - 1:
                break
        kk == jj
        lum[ii] = samp_arr[jj-1]+(samp_arr[jj]-samp_arr[jj-1])*(ranvec[ii]-CDF[jj-1])/(CDF[jj]-CDF[jj-1])
    return lum

### Calculate $\pi$

This is a classic test where you uniformly draw random points in a unit square with the origin at the left-bottom corner
and then measure how many points have a radius of 1 from the origin. This yields the area of a quarte circle...

In [ ]:
import random

nb.set_num_threads(4)

@njit(nb.float64(nb.int64),parallel=True, nogil=True, fastmath=True)
def monte_carlo_pi(nsamples):
    """
    parallel:
    enables the use of multiple threads
    while running the function. By default it will
    make use of pythons standard libraries like 
    multithreading/concurrent.futures to parallelize

    nogil:
    lifts the global interpreter lock. Useful
    if you prefer a manual thread mapping

    fastmath:
    lifts floating point accuracy restriction (IEEE 719)
    less strict accuracy requirements lead to speedup

    prange:
    numba substitue for the standard range() function.
    It will fallback to range() if parallel=False.
    Explicitely tells numba which loops to parallelize.
    """
    acc = 0
    for i in nb.prange(nsamples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [ ]:
%time monte_carlo_pi(1_000_000)

## The inevitable question

![spng](./6gemz6.jpg)

Ludwig kindly made a benchmark of this function with ***Julia***...

His implementation takes **3.2 ms** on this same machine (cast1) on the second function call.

The first function call, which is also used to compile the function in ***Julia*** (I think)  
takes **36 ms**

## The @jitclass decorator

A relatively new feature is the ***@jitclass*** decorator. By specifying the type of
class attributes, ***numba*** enables jitting of the class.  
All methods of the jitclass will be compiled in *nopython* mode.  
Since it is still in development, not all features of ***@jit*** are implemented yet.

In [75]:
from numba.experimental import jitclass

spec = [
    ("mass",    nb.float32[:]),
    ("pos",     nb.float32[:,:]),
    ("vel",     nb.float32[:,:]),
    ("acc",     nb.float32[:,:]),
    ("id",      nb.uint32[:]),
    ("active",  nb.boolean[:]),
    ("soft",    nb.float32),
    ("N",       nb.int64)
]

@jitclass(spec)
class Nbody:
    """
    spec:
    Contains the class attributes and their datatypes
    An __init__ method is required to initializw all 
    attributes given in the specification.
    """

    def __init__(self, N: int):
        self.mass   = np.zeros(N, dtype=np.float32)
        self.pos    = np.zeros((N,3), dtype=np.float32)
        self.vel    = np.zeros((N,3), dtype=np.float32)
        self.acc    = np.zeros((N,3), dtype=np.float32)
        self.id     = np.arange(N, dtype=np.uint32)
        self.active = np.zeros(N, dtype=nb.boolean)
        self.soft   = .1
        self.N      = N

    def kick(self, dt):
        self.vel += self.acc*dt

    def drift(self,dt):
        self.pos += self.vel*dt

    def calc_acc(self):
        """
        No parallel support yet. A possible solution
        would be to have a jitted function external to
        the class processing the arrays in parallel.
        """
        for i in range(self.N):
            self.acc[i] = np.zeros(3, dtype=np.float32)
            for j in range(self.N):
                if i != j:
                    dist_vec = self.pos[i]-self.pos[j]
                    dist = np.sqrt(dist_vec**2+self.soft)
                    self.acc[i] += self.mass[j] / dist**3 * dist_vec

    def _init_rand_pos(self, box=1.):
        np.random.seed(123)
        self.pos = np.random.uniform(0.,box,(self.N,3)).astype(np.float32)
        self.mass = np.ones_like(self.mass)
        self.acc = _calc_acc(self.mass, self.pos, self.soft, self.N)

    def init_rand_pos(self, box=1.):
        np.random.seed(123)
        self.pos = np.random.uniform(0.,box,(self.N,3)).astype(np.float32)
        self.mass = np.ones_like(self.mass)
        self.calc_acc()

nb.set_num_threads(4)

@njit(nb.float32[:,:](nb.float32[:],nb.float32[:,:], nb.float32, nb.int64),parallel=True)
def _calc_acc(mass_arr, pos_arr, soft, dim):
    acc_arr = np.zeros((dim,3),dtype=np.float32)
    for i in nb.prange(dim):
        # acc_arr[i] = np.zeros(3, dtype=np.float32)
        for j in range(dim):
            if i != j:
                dist_vec = pos_arr[i]-pos_arr[j]
                dist = np.sqrt(dist_vec**2 + soft)
                acc_arr[i] += mass_arr[j] / dist**3 * dist_vec
    return acc_arr
    



In [76]:
part = Nbody(4_000)

In [77]:
%time part.init_rand_pos()
a = part.acc

CPU times: user 5.51 s, sys: 0 ns, total: 5.51 s
Wall time: 5.5 s


In [78]:
%time part._init_rand_pos()
b = part.acc

CPU times: user 9.2 s, sys: 15.9 ms, total: 9.21 s
Wall time: 2.8 s


In [79]:
print(a-b)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


# Conclusion

***numba*** is a very handy and powerful tool to significantly improve numeric python and  
***numpy*** algorithms. It is especially useful, if your algorithms are already making heavy use  
of numpy's features. By only adhereing to basic principles of type stability within loops  
and a slightly unpythonic writing coding style, it can easily offer one to two orders of magnitude  
performance boost. It supports many native python and numpy functions and Additionally,  
it can benefit from other packages like ***scipy*** to improve functions used by `np.linalg`.  
There is also the possibility to ***jit*** an entire module with the ***@jit_module*** decorator  
and much more.

![das](./6gf5ud.jpg)

# Thank You

# APPENDIX

Stefan and Ilaria found an interesting edge case where numba really is more inefficient to use.

Apparently, it cannot speedup some native functions of numpy as there isn't anything to improve.

For example `np.sum` is already optimized to oblivion, numba only makes it perform worse by adding additional overhead.

This is however only true if those calls are standalone. As soon as the algorithms become  slightly more complicated,

numba can be considered.

In [ ]:
# This example was proposed by Stefan Heigl.
# It shows that native numpy function can be
# faster than their jitted counterparts.
# This only applies to single calls of
# numpy functions. Algoritms with more
# computations or iterations still benefit,
# even if native numpy functions are
# involved...

inp = np.ones((1024,1024,1024))

def sum_np(arr):
    return np.sum(arr)

@njit
def sum_nb(arr):
    return np.sum(arr)

@njit
def sum_nb2(arr):
    sum = 0.
    for i in range(arr.shape[0]):
        for j in range(arr.shape[0]):
            for k in range(arr.shape[0]):
                sum += arr[i,j,k]

    return sum

lol = inp.flatten()
@njit
def sum_nb3(arr):
    sum = 0.
    for i in range(lol.shape[0]):
        sum += arr[i]

    return sum